In [32]:
# reflected location error in ozone data simulation

import torch
import torch.fft
import numpy as np
import sys
import os
import logging
import argparse 
import pandas as pd
import pickle
import torch.optim as optim
import copy
import time
from sklearn.neighbors import BallTree
from typing import Optional, List, Tuple
from pathlib import Path

# --- CUSTOM PATHS ---
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# (필요 시 실제 GEMS_TCO 라이브러리 import)
try:
    from GEMS_TCO import kernels_for_simulation_no_trend_020626 as kernels_simulation
    
    
    
    from GEMS_TCO import orderings as _orderings
    from GEMS_TCO import alg_optimization, BaseLogger


except ImportError:
    print("Warning: GEMS_TCO modules not found. Ensure paths are correct.")
from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_gpu

from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle





In [29]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[-3,-1]      
lon_range_input=[121, 125] 

#lat_range_input=[-3,2]      
#lon_range_input=[121, 131] 

# 기존: df_map, ord_mm, nns_map, day_offsets = ...
# 수정 후: 변수명을 monthly_mean으로 변경

df_map, ord_mm, nns_map, monthly_mean = data_load_instance.load_maxmin_ordered_data_bymonthyear(
    lat_lon_resolution=lat_lon_resolution, 
    mm_cond_number=mm_cond_number,
    years_=years, 
    months_=month_range,
    lat_range=lat_range_input,   
    lon_range=lon_range_input
)

print(f"Global Monthly Mean: {monthly_mean}") # 확인용 출력


daily_aggregated_reg_vecc = [] 
daily_hourly_maps_reg_vecc = []      

daily_aggregated_irr_vecc = [] 
daily_hourly_maps_irr_vecc = []   


for day_index in range(31):
    hour_start_index = day_index * 8
    
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    monthly_mean,  # <--- 이 부분이 추가되어야 합니다
    hour_indices, 
    ord_mm=ord_mm,
    dtype=torch.float64, 
    keep_ori=False
    )

    daily_aggregated_reg_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_reg_vecc.append( day_hourly_map )

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    monthly_mean,  # <--- 이 부분이 추가되어야 합니다
    hour_indices, 
    ord_mm=ord_mm,
    dtype=torch.float64, 
    keep_ori= True
    )

    daily_aggregated_irr_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_irr_vecc.append( day_hourly_map )
print(daily_aggregated_irr_vecc[0].shape)

nn = daily_aggregated_irr_vecc[0].shape[0]


--- [AR(1) Analysis for Mean Function] ---
  Global Monthly Mean (a_y): 257.0667
  Fitted AR(1) Phi: 0.6765
Global Monthly Mean: {'2024_07_y24m07day01': 257.06668737814357, '2024_07_y24m07day02': 256.21248030343077, '2024_07_y24m07day03': 254.65984163444872, '2024_07_y24m07day04': 257.0527308280941, '2024_07_y24m07day05': 257.2790983012689, '2024_07_y24m07day06': 255.7956560813988, '2024_07_y24m07day07': 257.6881021700275, '2024_07_y24m07day08': 257.56118987013565, '2024_07_y24m07day09': 256.43433212608755, '2024_07_y24m07day10': 259.1536434348535, '2024_07_y24m07day11': 260.3522919246311, '2024_07_y24m07day12': 259.7040303955381, '2024_07_y24m07day13': 257.0934930401064, '2024_07_y24m07day14': 256.8174594747026, '2024_07_y24m07day15': 257.17490616699297, '2024_07_y24m07day16': 257.3539992400824, '2024_07_y24m07day17': 257.9564063039137, '2024_07_y24m07day18': 256.2079378650727, '2024_07_y24m07day19': 256.77166028450097, '2024_07_y24m07day20': 255.62018125292764, '2024_07_y24m07day21'

수정된 코드의 구조

Vecc_Irr (Original): 실제 위성 좌표 사용 (실수형).

Vecc_Reg (Snapped): 실제 좌표를 가장 가까운 격자점으로 이동시킨 데이터 (개수는 Irr와 동일, 좌표만 정수형 격자).

DW_Full (Complete): 위성 경로와 무관하게 직사각형 전체를 꽉 채운 데이터 (개수가 많음)

   [Simulation Info]
    - Daily Base (Offset): 265.0
    - Hourly Trend (Target for Dummies): [15. 10.  5.  0.  0.  5. 10. 15.]

In [37]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.neighbors import BallTree
import sys

# --- 1. CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64

# --- 2. GRID & FFT HELPERS ---
def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    if lat_start > lat_end and lat_step > 0: lat_step = -lat_step
    if lon_start > lon_end and lon_step > 0: lon_step = -lon_step
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lons = torch.arange(lon_start, lon_end + 0.0001, lon_step, device=device, dtype=dtype)
    lats = torch.round(lats * 10000) / 10000
    lons = torch.round(lons * 10000) / 10000
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')
    center_points = torch.stack([grid_lat.flatten(), grid_lon.flatten()], dim=1)
    return center_points, len(lats), len(lons)

def generate_high_res_field(target_lat_range, target_lon_range, t_steps, params, device, dtype):
    lat_res_factor, lon_res_factor = 200.0, 4.0
    lat_res_high = 0.044 / lat_res_factor
    lon_res_high = 0.063 / lon_res_factor
    
    t_lat_max, t_lat_min = max(target_lat_range), min(target_lat_range)
    lats_high = torch.arange(t_lat_max + 0.1, t_lat_min - 0.1, -lat_res_high, device=device, dtype=dtype)
    lons_high = torch.arange(target_lon_range[0] - 0.1, target_lon_range[1] + 0.1, lon_res_high, device=device, dtype=dtype)
    
    Nx, Ny, Nt = len(lats_high), len(lons_high), t_steps
    dlat, dlon, dt = lat_res_high, lon_res_high, 1.0
    
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    lags_x = torch.arange(Px, device=device, dtype=dtype) * dlat; lags_x[Px//2:] -= (Px * dlat)
    lags_y = torch.arange(Py, device=device, dtype=dtype) * dlon; lags_y[Py//2:] -= (Py * dlon)
    lags_t = torch.arange(Pt, device=device, dtype=dtype) * dt;   lags_t[Pt//2:] -= (Pt * dt)

    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    adv_lat, adv_lon = params[4], params[5]
    sigma_sq = phi1 / phi2 
    
    u_x = L_x - adv_lat * L_t
    u_y = L_y - adv_lon * L_t
    dist_sq = (u_x * torch.sqrt(phi3) * phi2)**2 + (u_y * phi2)**2 + (L_t * torch.sqrt(phi4) * phi2)**2
    C_vals = sigma_sq * torch.exp(-torch.sqrt(dist_sq + 1e-12))

    S = torch.fft.fftn(C_vals); S.real = torch.clamp(S.real, min=0)
    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=device, dtype=dtype))
    field_sim_raw = torch.fft.ifftn(torch.sqrt(S.real) * random_phase).real
    
    field_sim = field_sim_raw[:Nx, :Ny, :Nt]
    target_std = torch.sqrt(sigma_sq)
    field_calibrated = (field_sim - field_sim.mean()) * (target_std / (field_sim.std() + 1e-9))
    
    return field_calibrated, lats_high, lons_high

# --- [수정된 핵심 함수] 3가지 데이터셋 생성 ---
def generate_three_datasets(daily_maps_real, true_params_tensor, target_grid_info, device, dtype):
    
    lat_s, lat_e, lat_step, lon_s, lon_e, lon_step = target_grid_info
    
    # 1. Target Grid (Full)
    target_grid_coords, Nx, Ny = make_target_grid(lat_s, lat_e, lat_step, lon_s, lon_e, lon_step, device, dtype)
    full_grid_locs_np = target_grid_coords.cpu().numpy()
    target_tree = BallTree(np.radians(full_grid_locs_np), metric='haversine') 
    
    # 2. High-Res GP Field
    high_res_field, lats_high, lons_high = generate_high_res_field((lat_s, lat_e), (lon_s, lon_e), 8, true_params_tensor, device, dtype)
    
    hr_mesh_lat, hr_mesh_lon = torch.meshgrid(lats_high, lons_high, indexing='ij')
    hr_tree = BallTree(np.radians(torch.stack([hr_mesh_lat.flatten(), hr_mesh_lon.flatten()], dim=1).cpu().numpy()), metric='haversine')
    high_res_flat = high_res_field.reshape(-1, 8) 

    # 저장소 (3종류)
    # (1) Irr: Real Coordinates
    # (2) Reg: Snapped Coordinates (Subset of DW)
    # (3) DW : Full Grid Coordinates
    list_irr, list_reg, list_dw = [], [], []
    map_irr, map_reg, map_dw = {}, {}, {}

    noise_std = torch.sqrt(torch.exp(true_params_tensor[6]))
    DAILY_BASE = 260.0 + 5.0 
    HOURLY_TREND = torch.tensor([15.0, 10.0, 5.0, 0.0, 0.0, 5.0, 10.0, 15.0], device=device, dtype=dtype)

    day0_dict = daily_maps_real[0]
    sorted_keys = sorted([k for k in day0_dict.keys() if 'hm' in k or 'time' in k])
    
    # -------------------------------------------------------------
    # Step A: Generate [DW Full] & Cache Values for Reg
    # -------------------------------------------------------------
    # Full Grid에 대한 High-Res 매핑 인덱스 미리 계산
    _, hr_indices_full = hr_tree.query(np.radians(full_grid_locs_np), k=1)
    hr_indices_full = torch.tensor(hr_indices_full.flatten(), device=device)
    
    # 나중에 Reg 데이터 만들 때 쓸 값들을 임시 저장
    full_sim_values_cache = [] 

    for t_idx in range(8):
        key = f"time_{t_idx}"
        
        # GP + Noise + Trend + Base (Full Grid)
        gp_signal = high_res_flat[hr_indices_full, t_idx]
        current_trend = HOURLY_TREND[t_idx]
        sim_vals = gp_signal + (torch.randn_like(gp_signal) * noise_std) + DAILY_BASE + current_trend
        
        # Cache for Reg
        full_sim_values_cache.append(sim_vals)
        
        # Features for DW
        N_full = len(target_grid_coords)
        time_col = torch.full((N_full, 1), float(t_idx), device=device, dtype=dtype)
        dummies = F.one_hot(torch.tensor([t_idx]), num_classes=8).repeat(N_full, 1)[:, 1:].to(device=device, dtype=dtype)
        offset_col = torch.full((N_full, 1), DAILY_BASE, device=device, dtype=dtype)
        
        dw_row = torch.cat([target_grid_coords, sim_vals.unsqueeze(-1), time_col, dummies, offset_col], dim=1)
        list_dw.append(dw_row)
        map_dw[key] = dw_row

    # -------------------------------------------------------------
    # Step B: Generate [Vecc Irr] & [Vecc Reg] (Path-based)
    # -------------------------------------------------------------
    for t_idx, key in enumerate(sorted_keys):
        if t_idx >= 8: break
        
        real_tensor = day0_dict[key].to(device)
        real_locs = real_tensor[:, :2]
        N_points = len(real_locs)
        real_locs_np = real_locs.cpu().numpy()
        
        # 1. Vecc Irr (Original Coords)
        _, hr_indices_irr = hr_tree.query(np.radians(real_locs_np), k=1)
        gp_signal_irr = high_res_flat[torch.tensor(hr_indices_irr.flatten(), device=device), t_idx]
        
        current_trend = HOURLY_TREND[t_idx]
        sim_vals_irr = gp_signal_irr + (torch.randn_like(gp_signal_irr) * noise_std) + DAILY_BASE + current_trend
        
        dummies = F.one_hot(torch.tensor([t_idx]), num_classes=8).repeat(N_points, 1)[:, 1:].to(device=device, dtype=dtype)
        offset_col = torch.full((N_points, 1), DAILY_BASE, device=device, dtype=dtype)
        
        irr_row = torch.cat([real_locs, sim_vals_irr.unsqueeze(-1), real_tensor[:, 3].unsqueeze(-1), dummies, offset_col], dim=1)
        list_irr.append(irr_row)
        map_irr[key] = irr_row
        
        # 2. Vecc Reg (Snapped Coords)
        # 실제 위치에서 가장 가까운 "격자 인덱스" 찾기
        _, grid_indices = target_tree.query(np.radians(real_locs_np), k=1)
        grid_indices = grid_indices.flatten()
        
        # DW에서 생성해둔 값 가져오기 (Consistency 보장)
        sim_vals_reg = full_sim_values_cache[t_idx][grid_indices]
        mapped_locs = target_grid_coords[torch.tensor(grid_indices, device=device)]
        
        reg_row = torch.cat([mapped_locs, sim_vals_reg.unsqueeze(-1), real_tensor[:, 3].unsqueeze(-1), dummies, offset_col], dim=1)
        list_reg.append(reg_row)
        map_reg[key] = reg_row

    # Returns 3 Tuples
    return (
        (torch.cat(list_irr), map_irr),  # 1. Irr
        (torch.cat(list_reg), map_reg),  # 2. Reg
        (torch.cat(list_dw), map_dw)     # 3. DW
    )

# --- 4. EXECUTION ---
# ... (설정 부분 동일) ...
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64
target_grid_info = (-1.0, -3.0, 0.044, 121.0, 125.0, 0.063)

# True Params
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_range_time = 1.0
init_nugget    = 0.247
init_advec_lat = 0.0418
init_advec_lon = -0.1689

true_phi2 = 1.0 / init_range_lon              
true_phi1 = init_sigmasq * true_phi2          
true_phi3 = (init_range_lon / init_range_lat)**2
true_phi4 = (init_range_lon / init_range_time)**2

true_params_tensor = [
    torch.tensor([np.log(true_phi1)], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(true_phi2)], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(true_phi3)], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(true_phi4)], device=DEVICE, dtype=DTYPE),
    torch.tensor([init_advec_lat],    device=DEVICE, dtype=DTYPE),
    torch.tensor([init_advec_lon],    device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(init_nugget)], device=DEVICE, dtype=DTYPE)
]

print("Checking Inputs:")
print(f" -> Target SigmaSq: {init_sigmasq}")

if 'daily_hourly_maps_irr_vecc' in locals() and len(daily_hourly_maps_irr_vecc) > 0:
    print("\n🚀 Generating 3 Distinct Datasets (Irr, Reg, DW)...")
    
    # [수정] 3개를 명확히 받습니다.
    (vecc_irr_data, vecc_reg_data, dw_full_data) = generate_three_datasets(
        daily_hourly_maps_irr_vecc, true_params_tensor, target_grid_info, DEVICE, DTYPE
    )
    
    # Unpack
    agg_irr, map_irr = vecc_irr_data
    agg_reg, map_reg = vecc_reg_data
    agg_dw,  map_dw  = dw_full_data
    
    # Global Variables for Models
    daily_aggregated_irr_vecc_sim = [agg_irr]
    daily_hourly_maps_irr_vecc_sim = [map_irr]
    
    daily_aggregated_reg_vecc_sim = [agg_reg] # Snapped Swath
    daily_hourly_maps_reg_vecc_sim = [map_reg]
    
    
    daily_aggregated_dw_sim = [agg_dw] # Full Grid
    daily_hourly_maps_dw_sim = [map_dw]
    
    print(f"\n[Validation]")
    print(f"1. Vecc Irr (Swath, Float Coords): {agg_irr.shape}")
    print(f"2. Vecc Reg (Swath, Grid Coords):  {agg_reg.shape}")
    print(f"3. DW Full  (Rectangular Grid):    {agg_dw.shape}")
    
    # Grid Size Check
    _, Nx, Ny = make_target_grid(-1.0, -3.0, 0.044, 121.0, 125.0, 0.063, DEVICE, DTYPE)
    expected_dw = Nx * Ny * 8
    print(f"   -> DW Count Match: {agg_dw.shape[0]} == {expected_dw} (Nx{Nx}*Ny{Ny}*8)")

else:
    print("Error: Load data first.")


# 1. DW 데이터 가져오기
dw_tensor = daily_aggregated_dw_sim[0]
coords_dw = dw_tensor[:, :2].cpu().numpy()

print(f"Generating NNS Map for Full Grid (Size: {len(coords_dw)})...")

# 2. MaxMin Ordering 수행 (데이터 재정렬)
# Vecchia는 순서가 중요하므로, 무작위 격자 순서보다는 MaxMin이 성능이 좋습니다.
ord_mm_dw = _orderings.maxmin_cpp(coords_dw)
dw_tensor_ordered = dw_tensor[ord_mm_dw]  # ★ 중요: 데이터 순서 변경

# 3. DW용 NNS Map 생성
coords_dw_ordered = dw_tensor_ordered[:, :2].cpu().numpy()
nns_map_dw = _orderings.find_nns_l2(locs=coords_dw_ordered, max_nn=mm_cond_number)

Checking Inputs:
 -> Target SigmaSq: 13.059

🚀 Generating 3 Distinct Datasets (Irr, Reg, DW)...

[Validation]
1. Vecc Irr (Swath, Float Coords): torch.Size([22680, 12])
2. Vecc Reg (Swath, Grid Coords):  torch.Size([22680, 12])
3. DW Full  (Rectangular Grid):    torch.Size([23552, 12])
   -> DW Count Match: 23552 == 23552 (Nx46*Ny64*8)
Generating NNS Map for Full Grid (Size: 23552)...


# Fit vecchia max min time 2 

reg sim  에러나는 이유는 데이터 중복 snap 때문

중요! reg sim 은 데이터 중심, 실제 데이터 위치에서 가장 가까운 그리드로 이동 이래서 중복 가능, 콜레스키 깨지기도 

dw 는 그리드 중심, 타겟 레귤러 그리드에서 가장 가까운 점 가져오기 

In [ ]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
#mm_cond_number = 16   # Neighbors
nheads = 300           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 2
LBFGS_HISTORY_SIZE = 100 # 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 30    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

# [4.2042, 1.6348, 0.4721, -3.2695, 0.0218, -0.1689, -1.3984]
params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(daily_aggregated_irr_vecc_sim, torch.Tensor):
    daily_aggregated_irr_vecc_sim = daily_aggregated_irr_vecc_sim.to(DEVICE)

# Instantiate
model_instance = kernels_gpu.fit_vecchia_lbfgs(
    smooth=v,

    #input_map=daily_hourly_maps_reg_vecc_sim[0],
    #aggregated_data= daily_aggregated_reg_vecc_sim[0],
    #input_map=daily_hourly_maps_irr_vecc_sim[0],
    #aggregated_data= daily_aggregated_irr_vecc_sim[0],
    input_map=daily_hourly_maps_dw_sim[0],
    aggregated_data= daily_aggregated_dw_sim[0],
    nns_map=nns_map_dw,      # or nns_map for the swath data
    mm_cond_number=mm_cond_number,
    nheads=nheads
)

'''
model_instance = kernels_reparam_space_time_gpu_col.fit_vecchia_lbfgs(
    smooth=v,
    #input_map=daily_hourly_maps_vecc_sim[0],
    #aggregated_data= daily_aggregated_tensors_vecc_sim[0],

    input_map=daily_hourly_maps_irr_vecc_sim[0],
    aggregated_data= daily_aggregated_irr_vecc_sim[0],

    nns_map=None,
    mm_cond_number=mm_cond_number
)
''' 

# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
🚀 Pre-computing (Time Dummies & Latitude Centering, NO Offset)... [Mean Lat: -1.9900] ✅ Done. (Heads: 2400, Tails: 21152)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/2 / Loss: 1.681878 ---
  Param 0: Value=4.2081, Grad=-1.8621361352240146e-06
  Param 1: Value=1.6610, Grad=-1.1731858640516178e-06
  Param 2: Value=0.4206, Grad=-1.5414708073244188e-06
  Param 3: Value=-3.4774, Grad=-4.087361791672145e-07
  Param 4: Value=-0.0183, Grad=-6.413948392368363e-08
  Param 5: Value=-0.1815, Grad=4.270714601444002e-06
  Param 6: Value=-1.3861, Grad=-1.0950438231585916e-05
  Max Abs Grad: 1.095044e-05
------------------------------
--- Step 2/2 / Loss: 1.681606 ---
  Param 0: Value=4.2081, Grad=-1.8621361352240146e-06
  Param 1: Value=1.6610, Grad=-1.1731858640516178e-06
  Param 2: Value=0.4206, Grad=-1.5414708073244188e-06
  Param 3: Value=-3.4774, Grad=-4.087361791672145e-07
  Param

irr

Final Interpretable Params: {'sigma_sq': 13.251375803999347, 'range_lon': 0.17490332632183242, 'range_lat': 0.14574358781113017, 'range_time': 0.8728788765946908, 'advec_lat': -0.05107314841222374, 'advec_lon': -0.17666227434803639, 'nugget': 0.24422111544308162}

Optimization finished in 11.47s.
Results after 1 steps: [4.32762326002336, 1.7435218787174314, 0.3647691408733439, -3.2151268041425722, -0.05107314841222374, -0.17666227434803639, -1.4096812532177727, 1.1799846924366073]
Final Params: [ 4.32762326  1.74352188  0.36476914 -3.2151268  -0.05107315 -0.17666227
 -1.40968125]


reg

Final Interpretable Params: {'sigma_sq': 13.411637991321767, 'range_lon': 0.19734738984885475, 'range_lat': 0.16507470051912865, 'range_time': 0.9563503788327048, 'advec_lat': -0.049054522467388444, 'advec_lon': -0.17787447986717292, 'nugget': 0.5824048225534659}

Optimization finished in 12.16s.
Results after 1 steps: [4.218912539799084, 1.6227897029609746, 0.35713494830623127, -3.156317549877595, -0.049054522467388444, -0.17787447986717292, -0.5405895016506577, 1.2122487753198843]
Final Params: [ 4.21891254  1.6227897   0.35713495 -3.15631755 -0.04905452 -0.17787448
 -0.5405895 ]

# fit dw

difference data

In [5]:
day=0
daily_hourly_maps_reg_vecc_sim[day]['2024_07_y24m07day01_hm00:53'].shape

torch.Size([2835, 12])

In [9]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [-3, -1]
lon_range= [121, 125]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [daily_aggregated_dw_sim[day]]
daily_hourly_maps_dw = [daily_hourly_maps_dw_sim[day]]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(-3,-1,121,125)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([22680, 4])
22680


tensor([[ -2.9800, 121.0000,  -2.3671,   0.0000],
        [ -2.9800, 121.0630,   0.3398,   0.0000],
        [ -2.9800, 121.1260,   3.9098,   0.0000],
        [ -2.9800, 121.1890,   6.2922,   0.0000],
        [ -2.9800, 121.2520,   2.1264,   0.0000],
        [ -2.9800, 121.3150,   2.4780,   0.0000],
        [ -2.9800, 121.3780,  -3.4540,   0.0000],
        [ -2.9800, 121.4410,  -2.2484,   0.0000],
        [ -2.9800, 121.5040,   4.0353,   0.0000],
        [ -2.9800, 121.5670,   1.5458,   0.0000],
        [ -2.9800, 121.6300,  -3.3241,   0.0000],
        [ -2.9800, 121.6930,   1.8169,   0.0000],
        [ -2.9800, 121.7560,  -0.5641,   0.0000],
        [ -2.9800, 121.8190,   3.7625,   0.0000],
        [ -2.9800, 121.8820,  -6.0315,   0.0000],
        [ -2.9800, 121.9450,  -2.7170,   0.0000],
        [ -2.9800, 122.0080,   2.3653,   0.0000],
        [ -2.9800, 122.0710,  -3.4617,   0.0000],
        [ -2.9800, 122.1340,   7.0681,   0.0000],
        [ -2.9800, 122.1970,  -1.8939,   0.0000]],

In [10]:
from torch.nn import Parameter
dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 45x63, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.069512 | Max Grad: 1.137849e-02
  Params (Raw Log): log_phi1: 4.1809, log_phi2: 1.5701, log_phi3: 0.4999, log_phi4: -3.2947, advec_lat: 0.0773, advec_lon: -0.1356, log_nugget: -2.7566
--- Step 2/20 ---
 Loss: 1.028813 | Max Grad: 2.466505e-05
  Params (Raw Log): log_phi1: 4.1963, log_phi2: 1.5950, log_phi3: 0.5039, log_phi4: -3.2986, advec_lat: 0.0776, advec_lon: -0.1368, log_nugget: -4.1302
--- Step 3/20 ---
 Loss: 1.028628 | Max Grad: 2.466505e-05
  Params (Raw Log): log_p